# split -apply combine data analysis/cleanup

Wickham, Hadley (2011). "The split-apply-combine strategy for data analysis". Journal of Statistical Software. 40 (1): 1–29.



## split
Split data into groups based on key or a function

## Apply:
- Aggregation: Compute group statistics (mean, sum, count)
- Transformation: Normalize/fill NA data as per group.
- filter: Discard group/group members based on number of elements or some statistics

## Groupby
https://realpython.com/pandas-groupby/



In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                       'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.arange(5),
                      'data2' : np.arange(5) + 5})
df


key1 key2  data1  data2
0    a  one      0      5
1    a  two      1      6
2    b  one      2      7
3    b  two      3      8
4    a  one      4      9

How to split data into groups for some statistical analysis? Use groupby.

In [2]:
group_df = df['data1'].groupby(df['key1'])
type(group_df)

df.head()


key1 key2  data1  data2
0    a  one      0      5
1    a  two      1      6
2    b  one      2      7
3    b  two      3      8
4    a  one      4      9

Nothing is computed, but we can ask for specific values (stats).

## Apply and combine


# sum in elements in each group

In [3]:
group_df.sum()

key1
a    5
b    5
Name: data1, dtype: int32

# the count of elements in a group

In [4]:
group_df.count()

key1
a    3
b    2
Name: data1, dtype: int64

# using columns names

In [5]:
group_df =  df.groupby('key1')
print(group_df.count())
#print(df)

      key2  data1  data2
key1                    
a        3      3      3
b        2      2      2


In [6]:
# or subset groupby object before mean if only interested in data1
print(group_df[['data1']].mean())

         data1
key1          
a     1.666667
b     2.500000


In [7]:
# same as
df['data1'].groupby(df['key1']).mean()

key1
a    1.666667
b    2.500000
Name: data1, dtype: float64

# how to get a group

In [7]:
group_df.get_group('a')

key1 key2  data1  data2
0    a  one      0      5
1    a  two      1      6
4    a  one      4      9

# Can groupby multiple keys

In [8]:
df

key1 key2  data1  data2
0    a  one      0      5
1    a  two      1      6
2    b  one      2      7
3    b  two      3      8
4    a  one      4      9

In [9]:
df['data1'].groupby([df['key1'], df['key2']]).mean()

key1  key2
a     one     2.0
      two     1.0
b     one     2.0
      two     3.0
Name: data1, dtype: float64

# apply a custom function

In [10]:
df = pd.DataFrame({'animal': 'cat dog cat fish dog cat cat'.split(),
                     'size': list('SSMMMLL'),
                     'weight': [8, 10, 11, 1, 20, 12, 12],
                   'adult' : [False] * 5 + [True] * 2})
df

animal size  weight  adult
0    cat    S       8  False
1    dog    S      10  False
2    cat    M      11  False
3   fish    M       1  False
4    dog    M      20  False
5    cat    L      12   True
6    cat    L      12   True

In [11]:
group_df = df.groupby('animal')
type(group_df)
group_df.apply(lambda x: print(type(x)))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


Empty DataFrame
Columns: []
Index: []

In [12]:
group_df.apply(lambda x: x.min())


animal size  weight  adult
animal                           
cat       cat    L       8  False
dog       dog    M      10  False
fish     fish    M       1  False

# List the size of the animals with the highest weight

In [13]:
group_df['weight'].idxmax()

animal
cat     5
dog     4
fish    3
Name: weight, dtype: int64

In [14]:
df['size'][group_df['weight'].idxmax()]

5    L
4    M
3    M
Name: size, dtype: object

In [15]:
#find weight, idxmax, then index on size
def find_size_with_max_weight(x):
    return x['size'][ x['weight'].idxmax()]
group_df.apply(find_size_with_max_weight)
#find_size_with_max_weight(df)

animal
cat     L
dog     M
fish    M
dtype: object

# Groupby name and z-normalize weight

In [16]:
df

animal size  weight  adult
0    cat    S       8  False
1    dog    S      10  False
2    cat    M      11  False
3   fish    M       1  False
4    dog    M      20  False
5    cat    L      12   True
6    cat    L      12   True

In [17]:
# q maybe
# write code here
def z_normalize(x):
    m= x.mean()
    std= x.std(ddof=0)
    return (x-m)/std
    
df.groupby('animal')['weight'].apply(z_normalize)

animal   
cat     0   -1.677484
        2    0.152499
        5    0.762493
        6    0.762493
dog     1   -1.000000
        4    1.000000
fish    3         NaN
Name: weight, dtype: float64

# iterating over groupby object

In [18]:
for name, sub_df in group_df:
    print(name,'\n', sub_df)

cat 
   animal size  weight  adult
0    cat    S       8  False
2    cat    M      11  False
5    cat    L      12   True
6    cat    L      12   True
dog 
   animal size  weight  adult
1    dog    S      10  False
4    dog    M      20  False
fish 
   animal size  weight  adult
3   fish    M       1  False


# Groupby colums

In [19]:
df.dtypes

animal    object
size      object
weight     int64
adult       bool
dtype: object

In [20]:
# q try
# FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
#dtype_grouped = df.groupby(df.dtypes, axis=1)

dtype_grouped = df.groupby(df.dtypes, axis=1)

C:\Users\KOSEK\AppData\Local\Temp\ipykernel_31448\574497380.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  dtype_grouped = df.groupby(df.dtypes, axis=1)


In [21]:
for name, sub_df in dtype_grouped:
    print(name)
    print(sub_df)

bool
   adult
0  False
1  False
2  False
3  False
4  False
5   True
6   True
int64
   weight
0       8
1      10
2      11
3       1
4      20
5      12
6      12
object
  animal size
0    cat    S
1    dog    S
2    cat    M
3   fish    M
4    dog    M
5    cat    L
6    cat    L


# Grouping with functions

In [22]:
df1 = df[['animal', 'size', 'weight']]
df1= df1.set_index('animal')
df1

size  weight
animal             
cat       S       8
dog       S      10
cat       M      11
fish      M       1
dog       M      20
cat       L      12
cat       L      12

In [23]:
df1.groupby(len).sum()

size  weight
animal                
3       SSMMLL      73
4            M       1

# aggregate function

Aggregation means data transformation that produces scalar values from vector

In [24]:
# credit source: https://github.com/wesm/pydata-book 
tips = pd.read_csv('https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv')
tips.sample(5)

total_bill   tip smoker  day    time  size
93        16.32  4.30    Yes  Fri  Dinner     2
188       18.15  3.50    Yes  Sun  Dinner     3
18        16.97  3.50     No  Sun  Dinner     3
76        17.92  3.08    Yes  Sat  Dinner     2
5         25.29  4.71     No  Sun  Dinner     4

In [25]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

total_bill   tip smoker  day    time  size   tip_pct
0       16.99  1.01     No  Sun  Dinner     2  0.059447
1       10.34  1.66     No  Sun  Dinner     3  0.160542
2       21.01  3.50     No  Sun  Dinner     3  0.166587
3       23.68  3.31     No  Sun  Dinner     2  0.139780
4       24.59  3.61     No  Sun  Dinner     4  0.146808

In [26]:
grouped = tips.groupby(['day', 'smoker'])

# apply more than one function

In [27]:
grouped['tip'].agg(['mean', 'median'])

mean  median
day  smoker                  
Fri  No      2.812500   3.125
     Yes     2.714000   2.500
Sat  No      3.102889   2.750
     Yes     2.875476   2.690
Sun  No      3.167895   3.020
     Yes     3.516842   3.500
Thur No      2.673778   2.180
     Yes     3.030000   2.560

# Give a different name too. Handy if you use lambda function

In [29]:
grouped['tip'].agg([('group_mean','mean'), ('sample_size','count'), ('data_range', lambda x:x.max()- x.min())])

group_mean  sample_size  data_range
day  smoker                                     
Fri  No        2.812500            4        2.00
     Yes       2.714000           15        3.73
Sat  No        3.102889           45        8.00
     Yes       2.875476           42        9.00
Sun  No        3.167895           57        4.99
     Yes       3.516842           19        5.00
Thur No        2.673778           45        5.45
     Yes       3.030000           17        3.00

In [58]:
# if you want to make new columns as hierarchical row index
grouped[['tip','tip_pct']].agg([('group_mean','mean'), ('sample_size','count'), ('data_range', lambda x:x.max()- x.min())])

tip                           tip_pct              \
            group_mean sample_size data_range group_mean sample_size   
day  smoker                                                            
Fri  No       2.812500           4       2.00   0.151650           4   
     Yes      2.714000          15       3.73   0.174783          15   
Sat  No       3.102889          45       8.00   0.158048          45   
     Yes      2.875476          42       9.00   0.147906          42   
Sun  No       3.167895          57       4.99   0.160113          57   
     Yes      3.516842          19       5.00   0.187250          19   
Thur No       2.673778          45       5.45   0.160298          45   
     Yes      3.030000          17       3.00   0.163863          17   

                        
            data_range  
day  smoker             
Fri  No       0.067349  
     Yes      0.159925  
Sat  No       0.235193  
     Yes      0.290095  
Sun  No       0.193226  
     Yes      0.644685  
Thur No       0.193350  
     Yes      0.151240

In [30]:
# if you want to make new columns as hierarchical row index
grouped[['tip','tip_pct']].agg([('group_mean','mean'), ('sample_size','count'), ('data_range', lambda x:x.max()- x.min())]).stack()

tip    tip_pct
day  smoker                                  
Fri  No     group_mean    2.812500   0.151650
            sample_size   4.000000   4.000000
            data_range    2.000000   0.067349
     Yes    group_mean    2.714000   0.174783
            sample_size  15.000000  15.000000
            data_range    3.730000   0.159925
Sat  No     group_mean    3.102889   0.158048
            sample_size  45.000000  45.000000
            data_range    8.000000   0.235193
     Yes    group_mean    2.875476   0.147906
            sample_size  42.000000  42.000000
            data_range    9.000000   0.290095
Sun  No     group_mean    3.167895   0.160113
            sample_size  57.000000  57.000000
            data_range    4.990000   0.193226
     Yes    group_mean    3.516842   0.187250
            sample_size  19.000000  19.000000
            data_range    5.000000   0.644685
Thur No     group_mean    2.673778   0.160298
            sample_size  45.000000  45.000000
            data_range    5.450000   0.193350
     Yes    group_mean    3.030000   0.163863
            sample_size  17.000000  17.000000
            data_range    3.000000   0.151240

# Cross tabulation. Computing frequencies. 

Helpfull in running various statistical tests

In [31]:
tips.head()

total_bill   tip smoker  day    time  size   tip_pct
0       16.99  1.01     No  Sun  Dinner     2  0.059447
1       10.34  1.66     No  Sun  Dinner     3  0.160542
2       21.01  3.50     No  Sun  Dinner     3  0.166587
3       23.68  3.31     No  Sun  Dinner     2  0.139780
4       24.59  3.61     No  Sun  Dinner     4  0.146808

In [32]:
pd.crosstab(tips.day, tips.smoker)

smoker  No  Yes
day            
Fri      4   15
Sat     45   42
Sun     57   19
Thur    45   17

# Some examples

In [33]:
df = pd.DataFrame({'A' : [1, 1, 1, 2, 2, 2], 'B' : [1, 2, np.nan, 2, 2, np.nan]})
df


A    B
0  1  1.0
1  1  2.0
2  1  NaN
3  2  2.0
4  2  2.0
5  2  NaN

In [34]:
# group by A and  fill NA values in group by group mean
def fill_na(sub_df):
    return sub_df.fillna(sub_df.mean())
    
df.groupby('A').apply(fill_na)

A    B
A          
1 0  1  1.0
  1  1  2.0
  2  1  1.5
2 3  2  2.0
  4  2  2.0
  5  2  2.0

# fill negative values by group mean

In [35]:
#FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
#sub_series[mask]=sub_series[~mask].mean()

df = pd.DataFrame({'A' : [1, 1, 1, 2, 2, 2], 'B' : [1, 2, -1, 2, 2, -1]})
def fill_negative(sub_series):
    mask = sub_series < 0
    sub_series[mask]=sub_series[~mask].mean()
    return sub_series

df.groupby('A').transform(fill_negative)

C:\Users\KOSEK\AppData\Local\Temp\ipykernel_31448\1236342118.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sub_series[mask]=sub_series[~mask].mean()


B
0  1.0
1  2.0
2  1.5
3  2.0
4  2.0
5  2.0

# fill NA values by given values

In [36]:
df = pd.DataFrame({'A' : [1, 1, 1, 2, 2, 2], 'B' : [1, 2, np.nan, 2, 2, np.nan]})
df
grouped = df.groupby('A')
fill_values= {1:145, 2:22}
## hint see this
def  fill_given_values(sub_df):
    sub_df =sub_df.fillna(fill_values[sub_df.name])
    return sub_df
    
grouped.apply(fill_given_values)

A      B
A            
1 0  1    1.0
  1  1    2.0
  2  1  145.0
2 3  2    2.0
  4  2    2.0
  5  2   22.0

# Adding a columns

In [37]:
df = pd.DataFrame({'Color': 'Red Red Red Blue'.split(),
               'Value': [100, 150, 50, 50]})
df

Color  Value
0   Red    100
1   Red    150
2   Red     50
3  Blue     50

# add "length"  column in df based on length of color values

In [38]:
## write code here
df['length'] = df['Color'].apply(len)
df

Color  Value  length
0   Red    100       3
1   Red    150       3
2   Red     50       3
3  Blue     50       4

# computing exptected values
# compute the expexted value for each category

In [39]:
#np.random.seed(1)
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                   'b', 'b', 'b', 'b'],
                     'data': range(8),
                     'probability':[.1, .5, .1, .3, .5, .2, .2, .1]})
def compute_exp_val(sub_df):
    return sum(sub_df['data']*sub_df['probability'])
    
    
df.groupby('category').apply(compute_exp_val)

category
a    1.6
b    4.9
dtype: float64